In [1]:
from __future__ import division

import numpy as np
import scipy as sp
import scipy.sparse.linalg
from scipy.sparse import SparseEfficiencyWarning
import warnings

In [2]:
# dense cholesky
cholesky_dense = lambda x: sp.linalg.cholesky(x, lower=True)

# sparse cholesky
try:
    from sksparse.cholmod import cholesky as cholesky_sparse
except ImportError: # older versions
    from scikits.sparse.cholmod import cholesky as cholesky_sparse

In [3]:
# configs to try: seed=12, density=0.01, mul=0.5

In [3]:
seed = 12
mul1 = 0.1 # lower values ensure positive definitness
mul2 = 0.5

F = sp.sparse.rand(1000, 5, density=0.1, format='csc', random_state=seed)
S = F.dot(F.T)
S *= mul1
S = S+S.T

with warnings.catch_warnings():
    warnings.simplefilter('ignore', category=SparseEfficiencyWarning)
    S.setdiag(1 + mul2*np.random.RandomState(seed).rand(S.shape[0])) # keep 1's on diagonal
print 'density: ', S.data.size / np.prod(S.shape)
S

density:  0.049594


<1000x1000 sparse matrix of type '<type 'numpy.float64'>'
	with 49594 stored elements in Compressed Sparse Column format>

In [4]:
sp.sparse.linalg.eigsh(S, k=1, return_eigenvectors=False, which='SM')

array([ 0.74522417])

In [5]:
L_dense = cholesky_dense(S.A)

In [6]:
L_sparse = cholesky_sparse(S)

# check that $LL^{T} = S$

numpy/scipy

In [76]:
np.linalg.norm(S.A - L_dense.dot(L_dense.T)) / np.linalg.norm(S.A)

1.9320068704658742e-16

OK

cholmod

cholmod's representation is $LL^T = PSP^T$

In [95]:
PtL = L_sparse.apply_Pt(L_sparse.L())
np.linalg.norm(S.A - PtL.dot(PtL.T)) / np.linalg.norm(S.A)

2.3260732541161977e-16

OK

# check that $LL^Tx = y$ is equiv. to $Sx = y$

using the fact:
$LL^Tx = y$ $\rightarrow$
$
\begin{cases}
  L\omega &= y \\
  L^Tx    &= \omega \\
\end{cases}
$

In [7]:
x = np.random.RandomState(seed).rand(S.shape[0],)

In [8]:
y = S.dot(x)

## scipy dense

In [80]:
w1 = sp.linalg.solve_triangular(L_dense, y, lower=True, trans=0)

In [81]:
solx1 = sp.linalg.solve_triangular(L_dense, w1, lower=True, trans=1)

In [82]:
np.linalg.norm(x - solx1) / np.linalg.norm(x)

1.9004056755538052e-15

OK

## cholmod

In [9]:
w2 = L_sparse.solve_L(L_sparse.apply_P(y))

In [10]:
solx2 = L_sparse.apply_Pt(L_sparse.solve_Lt(w2))

In [11]:
np.linalg.norm(x - solx2) / np.linalg.norm(x)

0.43370385938462436

**NOT OK**

## scipy sparse (*replace cholmod's solve_L with scipy's spsolve_triangular*)

In [18]:
with warnings.catch_warnings():
    warnings.simplefilter('ignore', category=SparseEfficiencyWarning)
    w3 = sp.sparse.linalg.spsolve_triangular(L_sparse.L(), L_sparse.apply_P(y), lower=True)

In [19]:
solx3 = L_sparse.apply_Pt(sp.sparse.linalg.spsolve_triangular(L_sparse.L().T, w3, lower=False))

In [20]:
np.linalg.norm(x - solx3) / np.linalg.norm(x)

9.2043472086789351e-15

OK

## scipy vs cholmod

In [21]:
np.linalg.norm(w2-w3)/np.linalg.norm(w3)

0.11289535014654306

In [22]:
np.linalg.norm(solx2-solx3)/np.linalg.norm(solx3)

0.35965940159142373